In [22]:
words = open ('names.txt','r').read().splitlines()

In [23]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [24]:
len(words)

32033

In [25]:
min(len(w) for w in words)

2

In [26]:
max(len(w) for w in words)

15

In [27]:
b={}
for w in words:
    chs=['<S>']+ list(w) +['<E>']
    for ch1,ch2 in zip(chs,chs[1:]):
        biagram=(ch1,ch2)
        #print(biagram)
        b[biagram]=b.get(biagram,0)+1

In [28]:
sorted(b.items(),key=lambda kv: -kv[1])

[(('n', '<E>'), 6763),
 (('a', '<E>'), 6640),
 (('a', 'n'), 5438),
 (('<S>', 'a'), 4410),
 (('e', '<E>'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('<S>', 'k'), 2963),
 (('l', 'e'), 2921),
 (('e', 'n'), 2675),
 (('l', 'a'), 2623),
 (('m', 'a'), 2590),
 (('<S>', 'm'), 2538),
 (('a', 'l'), 2528),
 (('i', '<E>'), 2489),
 (('l', 'i'), 2480),
 (('i', 'a'), 2445),
 (('<S>', 'j'), 2422),
 (('o', 'n'), 2411),
 (('h', '<E>'), 2409),
 (('r', 'a'), 2356),
 (('a', 'h'), 2332),
 (('h', 'a'), 2244),
 (('y', 'a'), 2143),
 (('i', 'n'), 2126),
 (('<S>', 's'), 2055),
 (('a', 'y'), 2050),
 (('y', '<E>'), 2007),
 (('e', 'r'), 1958),
 (('n', 'n'), 1906),
 (('y', 'n'), 1826),
 (('k', 'a'), 1731),
 (('n', 'i'), 1725),
 (('r', 'e'), 1697),
 (('<S>', 'd'), 1690),
 (('i', 'e'), 1653),
 (('a', 'i'), 1650),
 (('<S>', 'r'), 1639),
 (('a', 'm'), 1634),
 (('l', 'y'), 1588),
 (('<S>', 'l'), 1572),
 (('<S>', 'c'), 1542),
 (('<S>', 'e'), 1531),
 (('j', 'a'), 1473),
 (

In [29]:
import torch

In [30]:
N=torch.zeros((27,27),dtype=torch.int32)

In [31]:
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}  #a:0 b:1
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

print(stoi)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [32]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1,ix2] += 1

In [33]:
p=N[0].float()
p=p/p.sum()
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [34]:
N[0]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [35]:
g=torch.Generator().manual_seed(2147483647)
p=torch.rand(3,generator=g)
p=p/p.sum()
torch.multinomial(p,num_samples=100,replacement=True,generator=g)

tensor([1, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 2, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0,
        0, 1, 1, 1])

In [36]:
P=(N+1).float()
P=P/P.sum(1,keepdim=True) #  分母 27*1

In [37]:
g=torch.Generator().manual_seed(2147483647)
for i in range(5):

    out=[]
    ix=0
    while True:
        p = P[ix]
        #p = p / p.sum()
        ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        out.append(itos[ix])
        if ix==0:
            break
    print(''.join(out))

junide.
janasah.
p.
cony.
a.


In [38]:
log_likelihood=0.0
n=0

for w in words:
    chs=['.']+ list(w) +['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob=P[ix1,ix2]
        logprob=torch.log(prob)
        log_likelihood += logprob
        n += 1

nll=-log_likelihood
nll=nll/n
print(f'{nll}')
print(f'{log_likelihood}')
#全部的是错的

2.4543561935424805
-559951.5625


In [39]:
xs,ys = [],[]

for w in words:
    chs=['.']+ list(w) +['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        print(ch1,ch2)
        xs.append(ix1)
        ys.append(ix2)

xs=torch.tensor(xs)
num=xs.nelement()
ys=torch.tensor(ys)
g=torch.Generator().manual_seed(2147483647)
W=torch.rand((27,27),generator=g,requires_grad=True)

. e
e m
m m
m a
a .
. o
o l
l i
i v
v i
i a
a .
. a
a v
v a
a .
. i
i s
s a
a b
b e
e l
l l
l a
a .
. s
s o
o p
p h
h i
i a
a .
. c
c h
h a
a r
r l
l o
o t
t t
t e
e .
. m
m i
i a
a .
. a
a m
m e
e l
l i
i a
a .
. h
h a
a r
r p
p e
e r
r .
. e
e v
v e
e l
l y
y n
n .
. a
a b
b i
i g
g a
a i
i l
l .
. e
e m
m i
i l
l y
y .
. e
e l
l i
i z
z a
a b
b e
e t
t h
h .
. m
m i
i l
l a
a .
. e
e l
l l
l a
a .
. a
a v
v e
e r
r y
y .
. s
s o
o f
f i
i a
a .
. c
c a
a m
m i
i l
l a
a .
. a
a r
r i
i a
a .
. s
s c
c a
a r
r l
l e
e t
t t
t .
. v
v i
i c
c t
t o
o r
r i
i a
a .
. m
m a
a d
d i
i s
s o
o n
n .
. l
l u
u n
n a
a .
. g
g r
r a
a c
c e
e .
. c
c h
h l
l o
o e
e .
. p
p e
e n
n e
e l
l o
o p
p e
e .
. l
l a
a y
y l
l a
a .
. r
r i
i l
l e
e y
y .
. z
z o
o e
e y
y .
. n
n o
o r
r a
a .
. l
l i
i l
l y
y .
. e
e l
l e
e a
a n
n o
o r
r .
. h
h a
a n
n n
n a
a h
h .
. l
l i
i l
l l
l i
i a
a n
n .
. a
a d
d d
d i
i s
s o
o n
n .
. a
a u
u b
b r
r e
e y
y .
. e
e l
l l
l i
i e
e .
. s
s t


In [40]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
%matplotlib inline
for k in range(100):
    xenc=F.one_hot(xs,num_classes=27).float()
    #plt.imshow(xenc)  ？？？错了
    logits=xenc @ W
    counts=logits.exp()
    probs=counts/counts.sum(1,keepdim=True)
    loss=-probs[torch.arange(num),ys].log().mean()
    print(loss.item())

    W.grad=None
    loss.backward()

    W.data += -100 * W.grad

    

3.359710931777954
2.9237558841705322
2.766462564468384
2.6875877380371094
2.6538374423980713
2.6204886436462402
2.6197128295898438
2.58722186088562
2.5786001682281494
2.563876152038574
2.571904182434082
2.548196315765381
2.5478415489196777
2.538696765899658
2.5501861572265625
2.5288281440734863
2.5311853885650635
2.525045394897461
2.53852915763855
2.517941474914551
2.5212442874908447
2.5175511837005615
2.532419204711914
2.5123236179351807
2.5154855251312256
2.5140812397003174
2.5293052196502686
2.5094664096832275
2.5113770961761475
2.5116212368011475
2.5265920162200928
2.5065829753875732
2.5074541568756104
2.5087223052978516
2.523876905441284
2.5036144256591797
2.50404691696167
2.5059943199157715
2.521503448486328
2.5010721683502197
2.5013046264648438
2.5037007331848145
2.519496202468872
2.4989867210388184
2.499116897583008
2.5018012523651123
2.5177927017211914
2.497263193130493
2.497347116470337
2.5002148151397705
2.5163371562957764
2.4958157539367676
2.4958834648132324
2.498875141143

In [41]:
g=torch.Generator().manual_seed(2147483647)
for i in range(5):

    out=[]
    ix=0
    while True:
        xenc=F.one_hot(xs,num_classes=27).float()
        logits=xenc @ W
        counts=logits.exp()
        probs=counts/counts.sum(1,keepdim=True)
        ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        out.append(itos[ix])
        if ix==0:
            break
    print(''.join(out))

.
.
nndnd.
ln.
snh.
